In [1]:
!pip install git+https://github.com/matterhorn-studio/MHSapi.git

  Cloning https://github.com/matterhorn-studio/MHSapi.git to /tmp/pip-req-build-vtqs5wmq
  Running command git clone --filter=blob:none --quiet https://github.com/matterhorn-studio/MHSapi.git /tmp/pip-req-build-vtqs5wmq
  Resolved https://github.com/matterhorn-studio/MHSapi.git to commit e6beb1788ddab337665968fb08389c6ac21627af
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.3/83.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.8/395.8 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 84.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.4/305.4 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58

In [3]:
import pydantic
pydantic.__version__

'2.4.2'

In [10]:
# Parameters, to be overwritten by papermill when run as nextflow pipeline
project_id = "1267"
opt_run_id = "481"
token = "1710e78f131e1f6d5cc903ff636821ce8988beb96a5838be4804e1d88caea969"
base_url = "https://matterhorn.studio/"

In [11]:
# Packages
import sys
print(sys.path)
import json

import numpy as np
import pandas as pd

['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/root/.ipython']


In [12]:
# 1. Initialise API client
import MHSapi
from MHSapi.MHSapi import MHSapiClient
from importlib.metadata import version

client = MHSapi.MHSapi.MHSapiClient(token=token, base_url=base_url)
print(version('MHSapi'))
print(MHSapi.__file__)
print(dir(MHSapi))
object_methods = [method_name for method_name in dir(client)
                  if callable(getattr(client, method_name))]
print(object_methods)

projects = client.experiments_list()
project = [p for p in projects if int(p.id) == int(project_id)][0]
parameters = client.parameters_list(project)


BASE URL: https://matterhorn.studio/
0.0.2
/usr/local/lib/python3.10/dist-packages/MHSapi/__init__.py
['MHSapi', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__']
['__class__', '__delattr__', '__dir__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'experiment_data', 'experiment_update_data', 'experiments_create', 'experiments_list', 'get_base_url', 'open_experiment', 'opt_run_list', 'parameters_create', 'parameters_list']
{}
[Experiment(id=1267, url='https://matterhorn.studio/experiments/1267/', name_text='Photonite-2024', data_table_json=''), Experiment(id=1196, url='https://matterhorn.studio/experiments/1196/', name_text='Photonite-2024', data_table_json=''), Experiment(id=1195, url='https://matterhorn.studio/exp

In [13]:
# 2. Download dataset
dataset = client.experiment_data(project)
print("Data")
print(dataset)

{"import_timestamp":{"0":1706707669.0541028976,"1":1706707669.0541028976,"2":1706707669.0541028976,"3":1706707669.0541028976,"4":1706707669.0541028976,"5":1706707669.0541028976,"6":1706707669.0541028976,"7":1706707669.0541028976,"8":1706707669.0541028976},"disabled":{"0":false,"1":false,"2":false,"3":false,"4":false,"5":false,"6":false,"7":false,"8":false},"Temperature (Celsius)":{"0":10.5,"1":12.3,"2":11.7,"3":25.5,"4":28.0,"5":15.0,"6":13.5,"7":25.0,"8":26.8},"Biomass Yield (g\/m3)":{"0":910.6302258842,"1":955.2308874596,"2":934.0850461644,"3":977.3216897534,"4":918.4279047088,"5":962.6146346605,"6":959.1759260206,"7":979.7014651398,"8":946.5538356882}}
Data
   import_timestamp  disabled  Temperature (Celsius)  Biomass Yield (g/m3)
0      1.706708e+09     False                   10.5            910.630226
1      1.706708e+09     False                   12.3            955.230887
2      1.706708e+09     False                   11.7            934.085046
3      1.706708e+09     False  

In [18]:
# 3. Get OptApp options

opt_runs = client.opt_run_list(project)
print(opt_runs)
opt_run = [p for p in opt_runs if int(p.id) == int(opt_run_id)][0]
print(opt_run)
#type = type(opt_run.run_options)
#method_list = [func for func in dir(type) if callable(getattr(type, func))]
#print(method_list)
run_options = opt_run.run_options.model_dump()

batch_size = 1
if 'batch_size' in run_options.keys():
    batch_size = run_options['batch_size']
batch_size

[OptimisationRun(id=480, experiment=1267, opt_app=None, run_options=X9fbc5f2e_3606_41ad_b934_5b6f019354eb_(root={})), OptimisationRun(id=481, experiment=1267, opt_app=1, run_options=X9fbc5f2e_3606_41ad_b934_5b6f019354eb_(root={'batch_size': L80ee1ee16_dfb2_49f2_8d92_0c7d0014a537_(root=L80ee1ee16_dfb2_49f2_8d92_0c7d0014a537_(root=3))}))]
id=481 experiment=1267 opt_app=1 run_options=X9fbc5f2e_3606_41ad_b934_5b6f019354eb_(root={'batch_size': L80ee1ee16_dfb2_49f2_8d92_0c7d0014a537_(root=L80ee1ee16_dfb2_49f2_8d92_0c7d0014a537_(root=3))})
<class 'str'> {"batch_size":3.0}


<ipython-input-18-2c1dc05749a9>:11: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  print(type(opt_run.run_options.json()), opt_run.run_options.json())
<ipython-input-18-2c1dc05749a9>:12: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  run_options = json.loads(opt_run.run_options.json())


3.0

In [28]:
opt_run.run_options.model_dump()['batch_size']

3

In [ ]:
|




# 4. Do BO
import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.utils import standardize
from gpytorch.mlls import ExactMarginalLogLikelihood

from botorch.models.transforms.input import Normalize
from botorch.models.transforms.outcome import Standardize

inputs = [p for p in parameters if p.outcome == False and p.timestamp == False]
outcome = [p for p in parameters if p.outcome == True][0]
X = dataset[[i.parameter_text for i in inputs]]
Y = dataset[[outcome.parameter_text]]
train_X = torch.tensor(X.to_numpy(dtype=np.float64))
train_Y = torch.tensor(Y.to_numpy(dtype=np.float64))
#train_Y = standardize(train_Y)
print(train_X.shape)
print(train_X)
print(train_Y.shape)
print(train_Y)

gp = SingleTaskGP(train_X, train_Y, input_transform=Normalize(d=train_X.shape[-1]), outcome_transform=Standardize(m=train_Y.shape[-1]))

mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_mll(mll)

#from botorch.acquisition import UpperConfidenceBound
#UCB = UpperConfidenceBound(gp, beta=0.1)

from botorch.optim import optimize_acqf

from botorch import fit_gpytorch_mll
from botorch.acquisition.monte_carlo import (
    qExpectedImprovement,
    qNoisyExpectedImprovement,
)
from botorch.sampling.normal import SobolQMCNormalSampler
from botorch.exceptions import BadInitialCandidatesWarning

upper_bounds = torch.tensor([p.upper_bound for p in inputs])
lower_bounds = torch.tensor([p.lower_bound for p in inputs])
bounds = torch.stack([lower_bounds, upper_bounds])
print(f"batch_size={batch_size}")

SMOKE_TEST = False
MC_SAMPLES = 256 if not SMOKE_TEST else 32
qmc_sampler = SobolQMCNormalSampler(sample_shape=torch.Size([MC_SAMPLES]))

qNEI = qNoisyExpectedImprovement(
    model=gp,
    X_baseline=train_X,
    sampler=qmc_sampler,
)
candidates, acq_value = optimize_acqf(
    acq_function=qNEI, bounds=bounds, q=int(batch_size), num_restarts=5, raw_samples=20,
)
candidates  # tensor([0.4887, 0.5063])
print("Candidates (raw):")
print(candidates.detach())

candidates = pd.DataFrame(candidates.numpy())
candidates.columns = [input.parameter_text for input in inputs]
candidates[outcome.parameter_text] = np.nan
candidates["opt_run_id"] = opt_run_id

print(candidates)


In [ ]:
# 5. Upload candidates
client.experiment_update_data(project, candidates)
